In [1]:
import numpy as np
import sympy as sp

**System**

![Full Diagram](Full_diagram_CV.jpg)

**Specify System**

In [2]:
P1 = 200     # kPa
P2 = 101.325 # kPa
Lp = 30      # m

# mass flow rate 
m = 30.84319738  #kg/s

# density of water at 95C
ρ = 961.9  #kg/m3

# viscosity water at 95C
μ = 0.2971

Vn = (m/ρ)*3600
print('The volumetric flowrate is {} m3/h'.format(Vn))

# Choose Vd = 1.15Vn
Vd = 1.15*Vn
print('The design volumetric flowrate is {} m3/h'.format(Vd))

The volumetric flowrate is 115.43352798419794 m3/h
The design volumetric flowrate is 132.7485571818276 m3/h


**MEB**

![Full eq](Full_form.png)

**Using MEB to find Delta TV**

In [3]:
ΔP_EP = P2-P1
ΔP_TV = -ΔP_EP

print('ΔP_TV = {} kPa'.format(ΔP_TV ))

ΔP_TV = 98.675 kPa


**Delta TV = Delta STV + Delta CV**

Choose Delta CV = 0.3Delta STV - RULE 2(d)

In [4]:
ΔP_STV = ΔP_TV/1.3
print('ΔP_STV = {} kPa'.format(ΔP_STV ))

ΔP_STV = 75.90384615384615 kPa


![Full Diagram](Delpf.png)

In [5]:
ΔP_f = ΔP_STV

#Assume f = 0.02 and that pipe diameter gets 25% bigger before and after control valve
f = 0.02
do = 0.25
d = sp.symbols('d')
L_gate_valve = 13*(1+do)*(d/1000)
L_reducer = (0.2*(1+do)*(d/1000))/f
L_enlarger = (0.125*(d/1000))/f
L_globe_valve = 340*(d/1000)
L_entrance = (0.5*(1+do)*(d/1000))/f

Lr = 2*L_gate_valve + L_reducer + L_enlarger + L_globe_valve + L_entrance

Le = Lp + Lr

equation = ((62544*f*Le*(Vd**2)*ρ)/(d**5)) - ΔP_STV

d_solution = sp.solve(equation,d)
print('Diameter is {} mm'.format(d_solution[0]))

Diameter is 117.321107379258 mm


**Choose Dnom**

In [6]:
dnom = 150 #mm
OD = 165.1 #mm
thickness = 4 #mm
ϵ = 0.046 #mm

ID = OD- 2*thickness
print('The inner diameter is {} mm'.format(ID))

Re = (354*Vd*ρ)/(ID*μ)
print('Reynolds number is {}'.format(Re))

f_real = 1/(-1.8*np.log10(((ϵ/ID)/3.7)**1.11 + 6.9/Re))**2

print('The real f is {}'.format(f_real))

The inner diameter is 157.1 mm
Reynolds number is 968465.5114490531
The real f is 0.015555563539544933


In [7]:
L_gate_valve_r = 13*(1+do)*(ID/1000)
L_reducer_r = (0.2*(1+do)*(ID/1000))/f_real
L_enlarger_r = (0.125*(ID/1000))/f_real
L_globe_valve_r = 340*(ID/1000)
L_entrance_r = (0.5*(1+do)*(ID/1000))/f_real

Lr_r = 2*L_gate_valve_r + L_reducer_r + L_enlarger_r + L_globe_valve_r + L_entrance_r
Le_r = Lp + Lr_r

ΔP_f_real = (62544*f_real*Le_r*(Vd**2)*ρ)/ID**5
print('The real ΔPf = {} kPa'. format(ΔP_f_real))

The real ΔPf = 16.99576626117652 kPa


**Now Delta CV**

In [8]:
ΔP_CV = ΔP_TV - ΔP_f_real
print('ΔP_CV = {} kPa'.format(ΔP_CV))
ΔP_CV_psi = ΔP_CV*0.145038
print('ΔP_CV_psi = {} psi'.format(ΔP_CV_psi))

C_cv = 270 #for single port
Vd_gal = Vd*4.40287
SG = ρ/1000

#Now check f(x)
fx = Vd_gal/(np.sqrt(ΔP_CV_psi/SG)*C_cv)
print('The f(x) = {}'.format(fx))

ΔP_CV = 81.67923373882348 kPa
ΔP_CV_psi = 11.846592703011481 psi
The f(x) = 0.616836609955863


**0.1 < f(x) < 1**

In [9]:
# Check velocity
Area1 = (np.pi/4)*(((1+do)*ID)/1000)**2
velocity1 = (Vn/3600)/Area1
print('Velocity just before heat exchanger is {} m/s'.format(velocity1))

Area2 = (np.pi/4)*((ID)/1000)**2
velocity2 = (Vn/3600)/Area2
print('Velocity at controlvalve is {} m/s'.format(velocity2))

Velocity just before heat exchanger is 1.0586860509495326 m/s
Velocity at controlvalve is 1.6541969546086448 m/s


**Velocity must be between 1 m/s and 5 m/s**